# PDI - Rebalancing data for June 2022

## Setting

In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [3]:
from defillama import DefiLlama
llama = DefiLlama()

In [4]:
import plotly.express as px
import plotly.graph_objects as go

In [5]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

## Data retrieving and filtering

### Token inclusion criteria

1. **Project and token characteristics** 
    1. The project must have a listing on Defi Llama
    1. The project’s token should have been listed on CoinGecko with pricing data at least 1 year prior to the date of inclusion in the index. 
    1. The project should have a token that is native to Ethereum L1 or L2. This excludes wrapped variants, where the underlying tokens are locked on an alt-L1.
    1. The project should be a going concern, with a dedicated team actively building, supporting and maintaining the project. 
    1. No rebasing or deflationary tokens.
    1. The project must be widely considered to be building a useful protocol or product. Projects that have ponzi characteristics at the core of their offering will not be considered.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project’s protocol or product must have significant usage.

1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The token should have at least $5mm of onchain liquidity on a single pair.
    1. The token must have shown consistent DeFi liquidity on Ethereum.

1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.

### Tokens list

#### DefiLlama

In [6]:
protocols = llama.get_all_protocols()

In [7]:
top_defi_tokens = {} 
for pr in protocols[:1000]:
    if 'Ethereum' in pr['chains'] and pr['address'] and pr['address']!='-' and pr['symbol'] != '-':
        symbol = ''.join(c for c in pr['symbol'] if c.isalnum()).upper()
        top_defi_tokens[symbol] = {
            'symbol': symbol, 
            'address': pr['address'].lower()
        }

In [8]:
top_defi_tokens['XSUSHI'] = {
    'symbol': 'XSUSHI',
    'address': "0x8798249c2e607446efb7ad49ec89dd1865ff4272",
}

#### CoinGecko

##### Tokens addresses

In [9]:
coins_list = pd.DataFrame(cg.get_coins_list(include_platform=True))

In [10]:
tokens = {} 
tokens_by_addr = {}
for index, coin in coins_list.iterrows():
    if len(coin['platforms']) >= 1 and 'ethereum' in list(coin['platforms'].keys()):
        symbol = ''.join(c for c in coin['symbol'] if c.isalnum()).upper()
        address = coin['platforms']['ethereum'].lower()
        token = {
            'symbol': symbol,
            'address': address 
        }
        tokens[symbol] = token
        tokens_by_addr[address] = token

In [11]:
tokens['USDC'], tokens['WETH']

({'symbol': 'USDC', 'address': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'},
 {'symbol': 'WETH', 'address': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'})

In [12]:
# These token's addresses cant be found in the coingecko list of eth addresses
exclude_symbols = []
for symb, token in top_defi_tokens.items():
    if token['address'] not in tokens_by_addr.keys():
        print(token)
        exclude_symbols.append(token['symbol'])

{'symbol': 'MULTI', 'address': '0x9fb9a33956351cf4fa040f65a13b835a3c8764e3'}
{'symbol': 'IB', 'address': 'fantom:0x00a35fd824c717879bf370e70ac6868b95870dfb'}
{'symbol': 'COW', 'address': 'heco:0x80861a817106665bca173db6ac2ab628a738c737'}
{'symbol': 'ABR', 'address': 'celo:0x6e512bfc33be36f2666754e996ff103ad1680cc9'}
{'symbol': 'FLAG', 'address': '0x57dd84042ec9507963016596a34edad42f7e4cce '}
{'symbol': 'LYRA', 'address': 'optimism:0x50c5725949a6f0c72e6c4a641f24049a917db0cb'}
{'symbol': 'SOLV', 'address': '0x256f2d67e52fe834726d2ddcd8413654f5eb8b53'}
{'symbol': 'SGT', 'address': '0x84810bcf08744d5862b8181f12d17bfd57d3b078'}
{'symbol': 'RSR', 'address': '0x8762db106b2c2a0bccb3a80d1ed41273552616e8'}
{'symbol': 'FHM', 'address': '0xfa1fbb8ef55a4855e5688c0ee13ac3f202486286'}
{'symbol': 'GRO', 'address': '0x09e64c2b61a5f1690ee6fbed9baf5d6990f8dfd0'}
{'symbol': 'SWPR', 'address': 'arbitrum:0xde903e2712288a1da82942dddf2c20529565ac30'}
{'symbol': 'HND', 'address': 'hundredfinance.jpg'}
{'symbol

In [13]:
for symb in exclude_symbols:
    del top_defi_tokens[symb]

In [14]:
# KEEP has this address in defillama, but etherscan says this address belongs to tBTC now
for token in top_defi_tokens.values():
    if token['address'] == "0x8daebade922df735c38c80c7ebd708af50815faa":
        print(token)
del top_defi_tokens['KEEP']

{'symbol': 'KEEP', 'address': '0x8daebade922df735c38c80c7ebd708af50815faa'}


##### Market data

In [15]:
cg_tokens_data = cg.get_coins_markets(vs_currency='USD', order='market_cap_desc')
for i in range(2, 20):
    cg_tokens_data.extend(cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', page=i))
    
cg_tokens_data = pd.DataFrame.from_records(cg_tokens_data)

In [16]:
tokens_market_data = cg_tokens_data.set_index(cg_tokens_data['symbol'].str.upper())

In [17]:
tokens_market_data.head()

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
BTC,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,29146.000,555175926134,1,6.120761e+11,2.648596e+10,30625.000,...,2.100000e+07,21000000.0,69045.00,-57.86656,2021-11-10T14:24:11.849Z,67.810000,42801.28648,2013-07-06T00:00:00.000Z,None,2022-05-23T23:28:50.886Z
ETH,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,1975.580,238319369663,2,NaN,1.461181e+10,2084.130,...,NaN,NaN,4878.26,-59.59110,2021-11-10T14:24:19.604Z,0.432979,455176.55096,2015-10-20T00:00:00.000Z,"{'times': 89.67699437920805, 'currency': 'btc'...",2022-05-23T23:28:39.241Z
USDT,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,1.004,73341181181,3,NaN,4.826826e+10,1.004,...,7.327509e+10,NaN,1.32,-24.35149,2018-07-24T00:00:00.000Z,0.572521,74.82361,2015-03-02T00:00:00.000Z,None,2022-05-23T23:27:06.994Z
BNB,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,318.370,53499205086,4,5.349921e+10,2.181182e+09,336.210,...,1.681370e+08,168137035.9,686.31,-53.71556,2021-05-10T07:24:17.097Z,0.039818,797668.81370,2017-10-19T00:00:00.000Z,None,2022-05-23T23:29:33.619Z
USDC,usd-coin,usdc,USD Coin,https://assets.coingecko.com/coins/images/6319...,1.004,53331951623,5,NaN,5.345133e+09,1.005,...,5.331484e+10,NaN,1.17,-14.56490,2019-05-08T00:40:28.300Z,0.891848,12.34007,2021-05-19T13:14:05.611Z,None,2022-05-23T23:29:47.847Z


### Liquidity check

In [18]:
dex_subgraph_urls = {
    'univ2': "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2",
    'sushi': "https://api.thegraph.com/subgraphs/name/zippoxer/sushiswap-subgraph-fork",
} 

In [19]:
base_tokens = [tokens['USDC'], tokens['WETH']]
base_tokens

[{'symbol': 'USDC', 'address': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'},
 {'symbol': 'WETH', 'address': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'}]

In [20]:
query = """
    query pairs ($token0_ids: [ID!]!, $token1_ids: [ID!]!) {
        pairs (first:1000, where: {token0_in: $token0_ids, token1_in: $token1_ids}){
            id
            token0 {
              id
            }
            token1 {
              id
            }
            reserveUSD
        }
    }
    """

In [21]:
base_tokens_ids = [token['address'] for token in base_tokens]
tokens_ids = [token['address'] for symb, token in top_defi_tokens.items()]

In [22]:
rows = {token['symbol']: {} for token in top_defi_tokens.values()}
for dex_name, url in dex_subgraph_urls.items():
    request = requests.post(url, json={'query': query, 'variables': {'token0_ids': base_tokens_ids, 'token1_ids': tokens_ids}})
    pairs1 = request.json()['data']['pairs']

    request = requests.post(url, json={'query': query, 'variables': {'token0_ids': tokens_ids, 'token1_ids': base_tokens_ids}})
    pairs2 = request.json()['data']['pairs']
    
    pairs = pairs1 + pairs2
    
    print(f"{dex_name}: {len(pairs)} pairs")
    
    for pair in pairs:
        if pair['token0']['id'] in base_tokens_ids:
            base_token = tokens_by_addr[pair['token0']['id']]
            token = tokens_by_addr[pair['token1']['id']]
        else:
            base_token = tokens_by_addr[pair['token1']['id']]
            token = tokens_by_addr[pair['token0']['id']]
        
        colname = f"{dex_name}_{base_token['symbol']}" 
        
        rows[token['symbol']][colname] = float(pair['reserveUSD'])

univ2: 272 pairs
sushi: 150 pairs


In [23]:
liquidities = pd.DataFrame.from_dict(rows, orient='index')
liq_check = (liquidities > 5e6).any(axis=1).sort_values(ascending=False)
liq_check_passed = liq_check[liq_check]

In [24]:
lq_passed = tokens_market_data[
    tokens_market_data.index.isin(liq_check_passed.index)
]

In [25]:
lq_passed

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
WBTC,wrapped-bitcoin,wbtc,Wrapped Bitcoin,https://assets.coingecko.com/coins/images/7598...,29123.000000,8041630925,13,8.041631e+09,274027491.0,30625.000000,...,2.765030e+05,2.765030e+05,70643.00,-58.77624,2021-11-10T14:40:19.650Z,3139.170000,827.69225,2019-04-02T00:00:00.000Z,None,2022-05-23T23:29:55.548Z
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,5.600000,2556598721,33,5.600563e+09,183515848.0,5.950000,...,1.000000e+09,1.000000e+09,44.92,-87.55725,2021-05-03T05:25:04.822Z,1.030000,442.53527,2020-09-17T01:20:38.214Z,None,2022-05-23T23:28:53.602Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,98.740000,1349404289,50,1.571223e+09,352144829.0,107.610000,...,1.600000e+07,1.600000e+07,661.69,-85.17951,2021-05-18T21:19:59.514Z,26.020000,276.84022,2020-11-05T09:20:11.928Z,None,2022-05-23T23:29:16.748Z
COMP,compound-governance-token,comp,Compound,https://assets.coingecko.com/coins/images/1077...,69.550000,476286369,111,6.946914e+08,54731533.0,75.100000,...,1.000000e+07,1.000000e+07,910.54,-92.37749,2021-05-12T02:29:08.794Z,54.910000,26.39303,2022-05-12T07:18:59.691Z,None,2022-05-23T23:30:09.826Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,1.180000,438530932,117,1.171430e+09,6808337.0,1.340000,...,1.000000e+09,1.000000e+09,7.30,-83.95933,2021-08-20T08:35:20.158Z,0.505506,131.73421,2021-03-27T09:10:29.155Z,None,2022-05-23T23:30:25.522Z
FXS,frax-share,fxs,Frax Share,https://assets.coingecko.com/coins/images/1342...,7.090000,425401035,121,7.023948e+08,35475165.0,8.230000,...,9.949117e+07,9.949117e+07,42.80,-83.49699,2022-01-12T15:22:27.465Z,1.500000,369.89336,2021-06-25T16:50:51.447Z,None,2022-05-23T23:30:25.532Z
YFI,yearn-finance,yfi,yearn.finance,https://assets.coingecko.com/coins/images/1184...,8939.340000,315788401,145,3.259668e+08,68737261.0,9790.590000,...,3.666600e+04,3.666600e+04,90787.00,-90.20166,2021-05-12T00:29:37.713Z,31.650000,28010.50198,2020-07-18T12:26:27.150Z,None,2022-05-23T23:29:38.397Z
SUSHI,sushi,sushi,Sushi,https://assets.coingecko.com/coins/images/1227...,1.520000,292658813,154,3.795061e+08,196903409.0,1.690000,...,2.422554e+08,2.500000e+08,23.38,-93.51153,2021-03-13T23:44:36.774Z,0.475381,219.06499,2020-11-04T14:53:53.560Z,None,2022-05-23T23:30:00.827Z
SYN,synapse-2,syn,Synapse,https://assets.coingecko.com/coins/images/1802...,1.220000,225226663,180,3.053786e+08,11548774.0,1.450000,...,NaN,2.500000e+08,4.92,-75.10300,2021-10-24T00:24:41.805Z,0.389600,214.11486,2021-09-21T21:19:58.123Z,None,2022-05-23T23:29:32.333Z


### Non quantative checks

In [28]:
exclude_list = [
    {
        'symbol': 'WBTC',
        'reason': 'Criteria 1.C'
    },
    {
        'symbol': 'XSUSHI',
        'reason': 'Duplicates SUSHI'
    },
    {
        'symbol': 'ERN',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'NFTX',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'FXS',
        'reason': 'Criteria 1.G'
    },
    {
        'symbol': 'RGT',
        'reason': 'Deprecated asset'
    },
    {
        'symbol': 'SUPER',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'BOND',
        'reason': 'Criteria 1.I'
    }
]
exclude_symbols = [token['symbol'] for token in exclude_list]

In [29]:
nq_passed = lq_passed[
    ~lq_passed.index.isin(exclude_symbols)
]

In [30]:
nq_passed

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,5.600000,2556598721,33,5.600563e+09,183515848.0,5.95000,...,1.000000e+09,1.000000e+09,44.92,-87.55725,2021-05-03T05:25:04.822Z,1.030000,442.53527,2020-09-17T01:20:38.214Z,None,2022-05-23T23:28:53.602Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,98.740000,1349404289,50,1.571223e+09,352144829.0,107.61000,...,1.600000e+07,1.600000e+07,661.69,-85.17951,2021-05-18T21:19:59.514Z,26.020000,276.84022,2020-11-05T09:20:11.928Z,None,2022-05-23T23:29:16.748Z
COMP,compound-governance-token,comp,Compound,https://assets.coingecko.com/coins/images/1077...,69.550000,476286369,111,6.946914e+08,54731533.0,75.10000,...,1.000000e+07,1.000000e+07,910.54,-92.37749,2021-05-12T02:29:08.794Z,54.910000,26.39303,2022-05-12T07:18:59.691Z,None,2022-05-23T23:30:09.826Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,1.180000,438530932,117,1.171430e+09,6808337.0,1.34000,...,1.000000e+09,1.000000e+09,7.30,-83.95933,2021-08-20T08:35:20.158Z,0.505506,131.73421,2021-03-27T09:10:29.155Z,None,2022-05-23T23:30:25.522Z
YFI,yearn-finance,yfi,yearn.finance,https://assets.coingecko.com/coins/images/1184...,8939.340000,315788401,145,3.259668e+08,68737261.0,9790.59000,...,3.666600e+04,3.666600e+04,90787.00,-90.20166,2021-05-12T00:29:37.713Z,31.650000,28010.50198,2020-07-18T12:26:27.150Z,None,2022-05-23T23:29:38.397Z
SUSHI,sushi,sushi,Sushi,https://assets.coingecko.com/coins/images/1227...,1.520000,292658813,154,3.795061e+08,196903409.0,1.69000,...,2.422554e+08,2.500000e+08,23.38,-93.51153,2021-03-13T23:44:36.774Z,0.475381,219.06499,2020-11-04T14:53:53.560Z,None,2022-05-23T23:30:00.827Z
SYN,synapse-2,syn,Synapse,https://assets.coingecko.com/coins/images/1802...,1.220000,225226663,180,3.053786e+08,11548774.0,1.45000,...,NaN,2.500000e+08,4.92,-75.10300,2021-10-24T00:24:41.805Z,0.389600,214.11486,2021-09-21T21:19:58.123Z,None,2022-05-23T23:29:32.333Z
TOKE,tokemak,toke,Tokemak,https://assets.coingecko.com/coins/images/1749...,4.360000,64328939,384,4.369780e+08,921501.0,4.81000,...,1.000000e+08,1.000000e+08,79.02,-94.47212,2021-11-04T00:35:29.799Z,4.360000,0.18431,2022-05-23T22:37:13.401Z,None,2022-05-23T23:29:23.043Z
ALCX,alchemix,alcx,Alchemix,https://assets.coingecko.com/coins/images/1411...,32.000000,41652409,502,7.630020e+07,2293093.0,34.73000,...,1.998380e+06,2.393060e+06,2066.20,-98.45507,2021-03-20T19:33:19.740Z,28.550000,11.80417,2022-05-12T22:34:00.644Z,None,2022-05-23T23:29:23.609Z


### Retrieve historical data & livetime check

In [31]:
exclude_list = []

prices_data = pd.DataFrame()
marketcaps = pd.DataFrame() 
for index, data in nq_passed[['id', 'symbol']].iterrows():
    id_ = data['id']
    symbol = data['symbol'].upper()
    data = cg.get_coin_market_chart_by_id(id_, vs_currency='USD', days='max')
  
    df_prices = pd.DataFrame(data['prices'], columns=['date', symbol])
    df_prices = df_prices[df_prices[symbol] > 0]
    df_prices['date'] = pd.to_datetime(df_prices['date'], unit='ms').dt.date
    df_prices['date'] = pd.to_datetime(df_prices['date'])
    df_prices = df_prices.set_index('date', drop=True)
    df_prices = df_prices.loc[~df_prices.index.duplicated(keep='first')]
    
    if len(df_prices) < 365:
        print(f'Excluding {symbol}, prices data available only for {len(df_prices)} < 365 days')
        exclude_list.append(symbol)
        continue
    prices_data = pd.concat([prices_data, df_prices], axis=1)
    
    df_mcaps = pd.DataFrame(data['market_caps'], columns=['date', symbol])
    df_mcaps = df_mcaps[df_mcaps[symbol] > 0]
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'], unit='ms').dt.date
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'])
    df_mcaps = df_mcaps.set_index('date', drop=True)
    df_mcaps = df_mcaps.loc[~df_mcaps.index.duplicated(keep='first')]
    
    if len(df_mcaps) < 365:
        print(f'Note: {symbol}, marketcap data available only for {len(df_mcaps)} < 365 days')
        ## continue
    df_mcaps = df_mcaps.reindex(df_prices.index)
    marketcaps = pd.concat([marketcaps, df_mcaps], axis=1)
    
    

Excluding SYN, prices data available only for 267 < 365 days
Excluding TOKE, prices data available only for 290 < 365 days
Excluding FLX, prices data available only for 162 < 365 days


In [32]:
lt_passed = nq_passed[
    ~nq_passed.index.isin(exclude_list)
]

In [33]:
lt_passed

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,5.60,2556598721,33,5.600563e+09,183515848.0,5.95,...,1.000000e+09,1.000000e+09,44.92,-87.55725,2021-05-03T05:25:04.822Z,1.030000,442.53527,2020-09-17T01:20:38.214Z,None,2022-05-23T23:28:53.602Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,98.74,1349404289,50,1.571223e+09,352144829.0,107.61,...,1.600000e+07,1.600000e+07,661.69,-85.17951,2021-05-18T21:19:59.514Z,26.020000,276.84022,2020-11-05T09:20:11.928Z,None,2022-05-23T23:29:16.748Z
COMP,compound-governance-token,comp,Compound,https://assets.coingecko.com/coins/images/1077...,69.55,476286369,111,6.946914e+08,54731533.0,75.10,...,1.000000e+07,1.000000e+07,910.54,-92.37749,2021-05-12T02:29:08.794Z,54.910000,26.39303,2022-05-12T07:18:59.691Z,None,2022-05-23T23:30:09.826Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,1.18,438530932,117,1.171430e+09,6808337.0,1.34,...,1.000000e+09,1.000000e+09,7.30,-83.95933,2021-08-20T08:35:20.158Z,0.505506,131.73421,2021-03-27T09:10:29.155Z,None,2022-05-23T23:30:25.522Z
YFI,yearn-finance,yfi,yearn.finance,https://assets.coingecko.com/coins/images/1184...,8939.34,315788401,145,3.259668e+08,68737261.0,9790.59,...,3.666600e+04,3.666600e+04,90787.00,-90.20166,2021-05-12T00:29:37.713Z,31.650000,28010.50198,2020-07-18T12:26:27.150Z,None,2022-05-23T23:29:38.397Z
SUSHI,sushi,sushi,Sushi,https://assets.coingecko.com/coins/images/1227...,1.52,292658813,154,3.795061e+08,196903409.0,1.69,...,2.422554e+08,2.500000e+08,23.38,-93.51153,2021-03-13T23:44:36.774Z,0.475381,219.06499,2020-11-04T14:53:53.560Z,None,2022-05-23T23:30:00.827Z
ALCX,alchemix,alcx,Alchemix,https://assets.coingecko.com/coins/images/1411...,32.00,41652409,502,7.630020e+07,2293093.0,34.73,...,1.998380e+06,2.393060e+06,2066.20,-98.45507,2021-03-20T19:33:19.740Z,28.550000,11.80417,2022-05-12T22:34:00.644Z,None,2022-05-23T23:29:23.609Z


### Marketcap ranking & top10 filtering

In [34]:
chosen_tokens = lt_passed.sort_values("market_cap", ascending=False).iloc[:10]

In [35]:
chosen_tokens

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,5.60,2556598721,33,5.600563e+09,183515848.0,5.95,...,1.000000e+09,1.000000e+09,44.92,-87.55725,2021-05-03T05:25:04.822Z,1.030000,442.53527,2020-09-17T01:20:38.214Z,None,2022-05-23T23:28:53.602Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,98.74,1349404289,50,1.571223e+09,352144829.0,107.61,...,1.600000e+07,1.600000e+07,661.69,-85.17951,2021-05-18T21:19:59.514Z,26.020000,276.84022,2020-11-05T09:20:11.928Z,None,2022-05-23T23:29:16.748Z
COMP,compound-governance-token,comp,Compound,https://assets.coingecko.com/coins/images/1077...,69.55,476286369,111,6.946914e+08,54731533.0,75.10,...,1.000000e+07,1.000000e+07,910.54,-92.37749,2021-05-12T02:29:08.794Z,54.910000,26.39303,2022-05-12T07:18:59.691Z,None,2022-05-23T23:30:09.826Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,1.18,438530932,117,1.171430e+09,6808337.0,1.34,...,1.000000e+09,1.000000e+09,7.30,-83.95933,2021-08-20T08:35:20.158Z,0.505506,131.73421,2021-03-27T09:10:29.155Z,None,2022-05-23T23:30:25.522Z
YFI,yearn-finance,yfi,yearn.finance,https://assets.coingecko.com/coins/images/1184...,8939.34,315788401,145,3.259668e+08,68737261.0,9790.59,...,3.666600e+04,3.666600e+04,90787.00,-90.20166,2021-05-12T00:29:37.713Z,31.650000,28010.50198,2020-07-18T12:26:27.150Z,None,2022-05-23T23:29:38.397Z
SUSHI,sushi,sushi,Sushi,https://assets.coingecko.com/coins/images/1227...,1.52,292658813,154,3.795061e+08,196903409.0,1.69,...,2.422554e+08,2.500000e+08,23.38,-93.51153,2021-03-13T23:44:36.774Z,0.475381,219.06499,2020-11-04T14:53:53.560Z,None,2022-05-23T23:30:00.827Z
ALCX,alchemix,alcx,Alchemix,https://assets.coingecko.com/coins/images/1411...,32.00,41652409,502,7.630020e+07,2293093.0,34.73,...,1.998380e+06,2.393060e+06,2066.20,-98.45507,2021-03-20T19:33:19.740Z,28.550000,11.80417,2022-05-12T22:34:00.644Z,None,2022-05-23T23:29:23.609Z


In [36]:
chosen_tokens_symbols = list(chosen_tokens.index)
prices_data = prices_data[chosen_tokens_symbols]
marketcaps = marketcaps[chosen_tokens_symbols]

In [37]:
without_nan_index = (marketcaps.isnull().sum(axis=1) == 0) & (prices_data.isnull().sum(axis=1) == 0)
marketcaps = marketcaps[without_nan_index]
prices_data = prices_data[without_nan_index]

In [38]:
prices_data

,UNI,AAVE,COMP,LDO,YFI,SUSHI,ALCX
date,,,,,,,
2021-03-02,25.592699,386.507922,494.358379,0.849448,34900.371111,18.798040,1050.297397
2021-03-03,25.673868,392.581993,501.438114,0.879645,33400.696195,17.925825,991.795750
2021-03-04,25.486634,397.858343,494.206140,0.836782,33881.025287,17.517664,1376.135328
2021-03-05,28.368215,376.610205,472.457882,0.687990,32560.148658,17.408828,1139.907512
2021-03-06,28.308459,369.647205,479.153858,0.783612,32137.453888,16.690631,1167.196998
...,...,...,...,...,...,...,...
2022-05-19,4.889555,81.728519,64.481027,1.370136,9083.386841,1.259283,30.955426
2022-05-20,5.196583,91.479758,67.672880,1.420394,9541.809737,1.370140,32.999642
2022-05-21,5.113348,90.366300,66.126189,1.319123,9200.355707,1.296737,31.688564


In [39]:
marketcaps

,UNI,AAVE,COMP,LDO,YFI,SUSHI,ALCX
date,,,,,,,
2021-03-02,7.871645e+09,4.751222e+09,2.142571e+09,1.181185e+07,1.237769e+09,2.362758e+09,3.338203e+07
2021-03-03,7.938771e+09,4.878411e+09,2.175709e+09,1.224257e+07,1.188119e+09,2.273568e+09,3.126962e+07
2021-03-04,7.933740e+09,4.948727e+09,2.154215e+09,1.185368e+07,1.210977e+09,2.240011e+09,4.418676e+07
2021-03-05,8.803390e+09,4.684718e+09,2.054969e+09,9.733795e+06,1.156134e+09,2.211941e+09,3.647704e+07
2021-03-06,8.703822e+09,4.580840e+09,2.077348e+09,1.101787e+07,1.153659e+09,2.123790e+09,3.716741e+07
...,...,...,...,...,...,...,...
2022-05-19,2.233139e+09,1.128029e+09,4.427092e+08,4.964952e+08,3.237477e+08,2.443990e+08,4.030301e+07
2022-05-20,2.372593e+09,1.256450e+09,4.639994e+08,5.174210e+08,3.389635e+08,2.640117e+08,4.293617e+07
2022-05-21,2.334569e+09,1.242585e+09,4.529641e+08,4.867826e+08,3.272163e+08,2.504307e+08,4.131932e+07


## Weighting 

**Weighting requirements**
* The maximum weight any one token can have is 30%.
* All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
* Any asset with a weight below 0.5% will be removed from the index.


### Weight mcaps

In [40]:
weights = marketcaps.div(marketcaps.sum(axis=1), axis=0)
weights = weights.sort_values(weights.last_valid_index(), axis=1, ascending=False)

In [41]:
fig = px.line(weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    ## showlegend=False,
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights without max constraint'
)

### Adjust weights

This process adjusts weights to adhere to the weight constraints described in our methodology. In addition the `remove_tiny_weights` function checks for any assets with a weight below the minimum threshold of $0.5\%$ and proportionally increases all remaining weights (which are $> 0.5\%$). Note that after this procedure readjusting weights to respect maximum may be needed.

In [42]:
def adjust_weights(weights, max_weight):
    w = weights.copy()
    
    while (w > max_weight).any(axis=None):
        w[w > max_weight] = max_weight
        c = 1 - w.sum(axis=1)
        w_less = w[w < max_weight]
        w[(w < max_weight) & (w > 0)] += w_less.div(w_less.sum(axis=1), axis=0).mul(c, axis=0)
    
    return w

In [43]:
def remove_tiny_weights(weights, min_weight):
    w = weights.copy()
    
    w[w < min_weight] = 0
    w = w.div(w.sum(axis=1), axis=0)
    
    return w

In [44]:
adjusted_weights = adjust_weights(weights, max_weight=0.3) 
adjusted_weights = remove_tiny_weights(adjusted_weights, min_weight=0.005)
adjusted_weights = adjust_weights(adjusted_weights, max_weight=0.3)

adjusted_weights = adjusted_weights.sort_values(adjusted_weights.last_valid_index(),ascending=False, axis = 1)
adjusted_weights

,UNI,AAVE,LDO,COMP,YFI,SUSHI,ALCX
date,,,,,,,
2021-03-02,0.3,0.30000,0.000000,0.149228,0.086209,0.164563,0.000000
2021-03-03,0.3,0.30000,0.000000,0.154377,0.084303,0.161320,0.000000
2021-03-04,0.3,0.30000,0.000000,0.153730,0.086418,0.159852,0.000000
2021-03-05,0.3,0.30000,0.000000,0.151573,0.085276,0.163151,0.000000
2021-03-06,0.3,0.30000,0.000000,0.155177,0.086178,0.158646,0.000000
...,...,...,...,...,...,...,...
2022-05-19,0.3,0.29511,0.129891,0.115820,0.084697,0.063939,0.010544
2022-05-20,0.3,0.30000,0.127183,0.114052,0.083318,0.064894,0.010554
2022-05-21,0.3,0.30000,0.124919,0.116241,0.083971,0.064266,0.010603


In [45]:
fig = px.line(adjusted_weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights with max constraint'
)

### Convert weights to the [1, 255] scale

On the contract side weights are integer numbers from the interval $[1, 255]$ with total sum $255$, so it's needed to convert retrieved weights to this format.

Note that one can't just round weights to integers after scaling, since it doesn't guarantee that their sum will be $255$. To fix that firstly floor function is applied to the weights and then $1$ is being added to the $k$ weights with largest fractional parts.

In [46]:
def convert_weights(weights):
    w_scaled = weights * 255
    w_res = np.floor(w_scaled).astype(int)    
    remainders = w_scaled - w_res    
    k = round(remainders.sum())
    w_below_max = w_res[w_res<76]
    for i in range(k):
        w_below_max[i] +=1
    for i in w_below_max.index:
        w_res[i] = w_below_max[i]
    return w_res

In [47]:
last_weights = adjusted_weights.iloc[-1].dropna()
last_weights

UNI      0.300000
AAVE     0.300000
LDO      0.120675
COMP     0.118707
YFI      0.081956
SUSHI    0.068245
ALCX     0.010418
Name: 2022-05-23 00:00:00, dtype: float64

In [48]:
converted_last_weights = convert_weights(last_weights)
converted_last_weights

UNI      76
AAVE     76
LDO      31
COMP     31
YFI      21
SUSHI    18
ALCX      2
Name: 2022-05-23 00:00:00, dtype: int64

## Summary table

In [49]:
pdi = pd.DataFrame()
pdi.index = chosen_tokens.index
pdi['name'] = chosen_tokens['name']
pdi['market_cap'] = marketcaps.iloc[-1].astype(int)
pdi['price'] = prices_data.iloc[-1]
pdi['weight'] = last_weights
pdi['weight_converted'] = converted_last_weights
pdi['address'] = [top_defi_tokens[symb]['address'] for symb in pdi.index]

pdi = pdi[pdi['weight'] > 0]
pdi = pdi.sort_values("market_cap",ascending=False)
pdi

,name,market_cap,price,weight,weight_converted,address
symbol,,,,,,
UNI,Uniswap,2521147054,5.512928,0.300000,76,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984
AAVE,Aave,1299371385,94.595402,0.300000,76,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9
LDO,Lido DAO,496175840,1.323604,0.120675,31,0x5a98fcbea516cf06857215779fd812ca3bef1b32
COMP,Compound,488084560,71.061189,0.118707,31,0xc00e94cb662c3520282e6f5717214004a7f26888
YFI,yearn.finance,336978469,9506.874677,0.081956,21,0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e
SUSHI,Sushi,280599895,1.455847,0.068245,18,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2
ALCX,Alchemix,42833508,32.974543,0.010418,2,0xdbdb4d16eda451d0503b854cf79d55697f90c8df
